In [3]:
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import json
data_dir = "./ddeess"
with open("ind_to_book.json", "r", encoding="utf-8") as json_file:
    ind_to_book = json.load(json_file)
def preprocess_text(text):
    tokens = text.lower().split()  
    return tokens
documents = []
file_paths = [os.path.join(data_dir, f"{i}.txt") for i in range(13629)]

for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as file:
        description = file.read()
        tokens = preprocess_text(description)
        tag = os.path.basename(file_path).split(".")[0]
        document = TaggedDocument(words=tokens, tags=[tag])
        documents.append(document)

model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=10)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

k = 10  
book_vectors = [model.dv[f"{i}"] for i in range(13629)]  
nbrs = NearestNeighbors(n_neighbors=k, metric="cosine").fit(book_vectors)



Recommended books for  Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)) (['0451205480']):
 Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)) (['0451205480'])
Belgrave Square (['0449222276', '0449906787'])
Greenwich Killing Time (['0425104974', '0970238304'])
Tombstone Courage: A Joanna Brady Mystery (['0380765462'])
SS/GB (['0345284941'])


In [16]:
user_input_index = 12475

distances, indices = nbrs.kneighbors([book_vectors[user_input_index]])

recommended_book_indices = indices[0]

In [18]:
print(f"Recommended books for {ind_to_book[str(user_input_index)]['title']} ({ind_to_book[str(user_input_index)]['isbns']}):")
for book_index in recommended_book_indices:
    print(f"{ind_to_book[str(book_index)]['title']}")
    print(book_index)

Recommended books for Three Act Tragedy (Hercule Poirot Mysteries (Paperback)) (['0425091805']):
Three Act Tragedy (Hercule Poirot Mysteries (Paperback))
12475
The Passenger
11385
Sudden Mischief
9192
Caught In The Act
1907
Bite (Love Spell)
1336


In [14]:
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors
from gensim.models.doc2vec import Doc2Vec
import json
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-100")

data_dir = "./ddeess"

with open("ind_to_book.json", "r", encoding="utf-8") as json_file:
    ind_to_book = json.load(json_file)

k = 5  

def get_document_vector(book_index):
    with open(os.path.join(data_dir, f"{book_index}.txt"), "r", encoding="utf-8") as file:
        description = file.read()

    title = ind_to_book.get(str(book_index), {}).get("title", "")

    combined_text = title + " " + description

    return model.infer_vector(combined_text.lower().split())

user_input_index = 0  

user_input_vector = get_document_vector(user_input_index)

book_vectors = [get_document_vector(i) for i in range(13629)]

nbrs = NearestNeighbors(n_neighbors=k, metric="cosine").fit(book_vectors)
distances, indices = nbrs.kneighbors([user_input_vector])

recommended_book_indices = indices[0]

print(f"Recommended books for {ind_to_book.get(str(user_input_index), {}).get('title', 'Unknown Title')} ({ind_to_book.get(str(user_input_index), {}).get('isbns', 'Unknown ISBNs')}):")
for book_index in recommended_book_indices:
    book_title = ind_to_book.get(str(book_index), {}).get("title", "Unknown Title")
    book_isbns = ind_to_book.get(str(book_index), {}).get("isbns", "Unknown ISBNs")
    similarity = 1 - distances[recommended_book_indices.tolist().index(book_index)]
    print(f"{book_title} ({book_isbns}), Similarity: {similarity}")


ValueError: unable to read local cache '/home/virendrasinh10/gensim-data/information.json' during fallback, connect to the Internet and retry